# Capstone Project - The Optimal Locations for Indian Restaurent

## Applied Data Science Capstone by IBM/Coursera



## 1. Importing Libraries

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import pandas.io.json
import requests
import json

from bs4 import BeautifulSoup

from geopy.geocoders import Nominatim

import geocoder
import folium

print('Libraries Imported')

Libraries Imported



## 2. Scrapping Data from Wikipedia Paga
   We will work in Borough of London, UK, we will scrap the data from [List of London Borough](https://en.wikipedia.org/wiki/List_of_London_boroughs). Page contens two table 'List of boroughs and local authorities' and
   'City of London'

In [2]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_London_boroughs',header=0)[0]

In [3]:
df

,Borough,Inner,Status,Local authority,Political control,Headquarters,Area (sq mi),Population (2013 est)[1],Co-ordinates,Nr. in map
0,Barking and Dagenham [note 1],NaN,NaN,Barking and Dagenham London Borough Council,Labour,"Town Hall, 1 Town Square",13.93,194352,51°33′39″N 0°09′21″E﻿ / ﻿51.5607°N 0.1557°E,25
1,Barnet,NaN,NaN,Barnet London Borough Council,Conservative,"Barnet House, 2 Bristol Avenue, Colindale",33.49,369088,51°37′31″N 0°09′06″W﻿ / ﻿51.6252°N 0.1517°W,31
2,Bexley,NaN,NaN,Bexley London Borough Council,Conservative,"Civic Offices, 2 Watling Street",23.38,236687,51°27′18″N 0°09′02″E﻿ / ﻿51.4549°N 0.1505°E,23
3,Brent,NaN,NaN,Brent London Borough Council,Labour,"Brent Civic Centre, Engineers Way",16.70,317264,51°33′32″N 0°16′54″W﻿ / ﻿51.5588°N 0.2817°W,12
4,Bromley,NaN,NaN,Bromley London Borough Council,Conservative,"Civic Centre, Stockwell Close",57.97,317899,51°24′14″N 0°01′11″E﻿ / ﻿51.4039°N 0.0198°E,20
5,Camden,NaN,NaN,Camden London Borough Council,Labour,"Camden Town Hall, Judd Street",8.40,229719,51°31′44″N 0°07′32″W﻿ / ﻿51.5290°N 0.1255°W,11
6,Croydon,NaN,NaN,Croydon London Borough Council,Labour,"Bernard Weatherill House, Mint Walk",33.41,372752,51°22′17″N 0°05′52″W﻿ / ﻿51.3714°N 0.0977°W,19
7,Ealing,NaN,NaN,Ealing London Borough Council,Labour,"Perceval House, 14-16 Uxbridge Road",21.44,342494,51°30′47″N 0°18′32″W﻿ / ﻿51.5130°N 0.3089°W,13
8,Enfield,NaN,NaN,Enfield London Borough Council,Labour,"Civic Centre, Silver Street",31.74,320524,51°39′14″N 0°04′48″W﻿ / ﻿51.6538°N 0.0799°W,30
9,Greenwich [note 2],[note 3],Royal,Greenwich London Borough Council,Labour,"Woolwich Town Hall, Wellington Street",18.28,264008,51°29′21″N 0°03′53″E﻿ / ﻿51.4892°N 0.0648°E,22


In [4]:
df2 = pd.read_html('https://en.wikipedia.org/wiki/List_of_London_boroughs',header=0)[1]

In [5]:
df = df.append(df2, ignore_index=True)

C:\Users\Shrirang\Anaconda3\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [6]:
df

,Area (sq mi),Borough,Co-ordinates,Headquarters,Inner,Local authority,Nr. in map,Nr. inmap,Political control,Population (2013 est)[1],Population(2011 est),Status
0,13.93,Barking and Dagenham [note 1],51°33′39″N 0°09′21″E﻿ / ﻿51.5607°N 0.1557°E,"Town Hall, 1 Town Square",NaN,Barking and Dagenham London Borough Council,25.0,NaN,Labour,194352.0,NaN,NaN
1,33.49,Barnet,51°37′31″N 0°09′06″W﻿ / ﻿51.6252°N 0.1517°W,"Barnet House, 2 Bristol Avenue, Colindale",NaN,Barnet London Borough Council,31.0,NaN,Conservative,369088.0,NaN,NaN
2,23.38,Bexley,51°27′18″N 0°09′02″E﻿ / ﻿51.4549°N 0.1505°E,"Civic Offices, 2 Watling Street",NaN,Bexley London Borough Council,23.0,NaN,Conservative,236687.0,NaN,NaN
3,16.70,Brent,51°33′32″N 0°16′54″W﻿ / ﻿51.5588°N 0.2817°W,"Brent Civic Centre, Engineers Way",NaN,Brent London Borough Council,12.0,NaN,Labour,317264.0,NaN,NaN
4,57.97,Bromley,51°24′14″N 0°01′11″E﻿ / ﻿51.4039°N 0.0198°E,"Civic Centre, Stockwell Close",NaN,Bromley London Borough Council,20.0,NaN,Conservative,317899.0,NaN,NaN
5,8.40,Camden,51°31′44″N 0°07′32″W﻿ / ﻿51.5290°N 0.1255°W,"Camden Town Hall, Judd Street",NaN,Camden London Borough Council,11.0,NaN,Labour,229719.0,NaN,NaN
6,33.41,Croydon,51°22′17″N 0°05′52″W﻿ / ﻿51.3714°N 0.0977°W,"Bernard Weatherill House, Mint Walk",NaN,Croydon London Borough Council,19.0,NaN,Labour,372752.0,NaN,NaN
7,21.44,Ealing,51°30′47″N 0°18′32″W﻿ / ﻿51.5130°N 0.3089°W,"Perceval House, 14-16 Uxbridge Road",NaN,Ealing London Borough Council,13.0,NaN,Labour,342494.0,NaN,NaN
8,31.74,Enfield,51°39′14″N 0°04′48″W﻿ / ﻿51.6538°N 0.0799°W,"Civic Centre, Silver Street",NaN,Enfield London Borough Council,30.0,NaN,Labour,320524.0,NaN,NaN
9,18.28,Greenwich [note 2],51°29′21″N 0°03′53″E﻿ / ﻿51.4892°N 0.0648°E,"Woolwich Town Hall, Wellington Street",[note 3],Greenwich London Borough Council,22.0,NaN,Labour,264008.0,NaN,Royal


## 3 Plot the Borough on Map

In [7]:
# define a function to get coordinates
def get_latlng1(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, London,UK '.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [14]:
loaded = False
try:
    London_df = pd.read_csv('London_df.csv')
    loaded = True
except:
    pass
    

In [15]:
if not loaded:  
    coords_london = [ get_latlng1(neighborhood) for neighborhood in df["Borough"].tolist() ]
    df['LatLng'] = coords_london
    df['Lat'] = df['LatLng'].apply(lambda x:x[0])
    df['Lng'] = df['LatLng'].apply(lambda x:x[1])
    London_df = df[['Borough','Lat','Lng']]
    London_df.Borough = London_df['Borough'].apply(lambda x:x.split('[')[0])
    London_df.to_csv(r'C:\Users\Shrirang\Desktop\Python Practice\Online Courses\IMB Data Science Cousre\Capstone_Project\Coursera_Capstone\Capstone_project\London_df.csv')

In [16]:
London_df

,Unnamed: 0,Borough,Lat,Lng
0,0,Barking and Dagenham,51.574790,0.163640
1,1,Barnet,51.627300,-0.253760
2,2,Bexley,51.452078,0.069931
3,3,Brent,51.609783,-0.194672
4,4,Bromley,51.601511,-0.066365
5,5,Camden,51.532360,-0.127960
6,6,Croydon,51.593470,-0.083380
7,7,Ealing,51.514060,-0.300730
8,8,Enfield,51.540024,-0.077502
9,9,Greenwich,51.484540,0.002750


In [17]:
#first we need to get the co-ordinated of London,UK
g = geocoder.arcgis( 'London, UK ')
coordinated_centerlondon = g.latlng

In [18]:
coordinated_centerlondon

[51.50642000000005, -0.1272099999999341]

In [19]:
#Create map of London and superinpose the Neighborhood on it
map_la = folium.Map(location = coordinated_centerlondon, zoom_start = 10)
folium.Marker(coordinated_centerlondon, popup='London_center').add_to(map_la)
for lat, lng in zip(London_df['Lat'], London_df['Lng']):
    #folium.CircleMarker([lat, lng], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_la)
    folium.Circle([lat,lng], radius=30,color ="blue").add_to(map_la)
map_la

## Data from Foursquere API

In [29]:
client_Id = '0TW3ZXMUWOAWPJ0SMUVEXLCPDV3S2VFDGNGGOM0PQVSTVTN1'
client_Secret = 'J4QS4JI0ZXJ1SBKJO1WKHMRIUZKUVUSBAOYCW2OC2LWUHTLO'


In [28]:
food_category = '4d4b7105d754a06374d81259'
indian_Restaurant =     ['4bf58dd8d48988d10f941735','54135bf5e4b08f3d2429dfe5','54135bf5e4b08f3d2429dff3',
                         '54135bf5e4b08f3d2429dff5','54135bf5e4b08f3d2429dfe2','54135bf5e4b08f3d2429dff2',
                         '54135bf5e4b08f3d2429dfe1','54135bf5e4b08f3d2429dfe3','54135bf5e4b08f3d2429dfe8',
                         '54135bf5e4b08f3d2429dfe9','54135bf5e4b08f3d2429dfe6','54135bf5e4b08f3d2429dfdf',
                         '54135bf5e4b08f3d2429dfe4','54135bf5e4b08f3d2429dfe7','54135bf5e4b08f3d2429dfea',
                         '54135bf5e4b08f3d2429dfeb','54135bf5e4b08f3d2429dfed','54135bf5e4b08f3d2429dfee',
                         '54135bf5e4b08f3d2429dff4','54135bf5e4b08f3d2429dfe0','54135bf5e4b08f3d2429dfdd',
                         '54135bf5e4b08f3d2429dff6','54135bf5e4b08f3d2429dfef','54135bf5e4b08f3d2429dff0',
                         '54135bf5e4b08f3d2429dff1','54135bf5e4b08f3d2429dfde','54135bf5e4b08f3d2429dfec']